In [ ]:
import numpy as np 
import pandas as pd
import pyfmi 

In [ ]:
def cost_of_comfort(ts, csp, hsp, idf_steps_per_hour=6):
    model = pyfmi.load_fmu('model.fmu')
    
    # define time period of interest
    te_of_interest = ts + 86400.*(30) # 30 days from current time
    
    # simulate one more day for slicing: we have to start EP simulation from midnight and end at midnight
    day = ts // 86400.
    ts_midnight = 86400. * day # starting from midnight of the same day
    te = ts_midnight + 86400.*(30+1) # 30 days, with 1 day for floating

    # simulation options
    options = model.simulate_options()
    dt = 3600./idf_steps_per_hour
    ncp = (te - ts_midnight)/dt
    options['ncp'] = int(ncp)

    # construct constant inputs
    t = np.linspace(ts_midnight, te+1, int(ncp)+1)
    csp = csp*np.ones_like(t)
    hsp = hsp*np.ones_like(t)
    u_traj = np.vstack((t, csp, hsp)).T
    res = model.simulate(ts_midnight, te, input=(['csp', 'hsp'], u_traj), options=options)

    # get energy consumption
    power_in_W = res['hvac_power']

    # slice out the period of interest
    period_of_interest = np.logical_and(t >= ts, t <= te_of_interest)
    power_of_interest = power_in_W[period_of_interest]
    total_energy = power_of_interest.sum() * dt / 3600./1000. # kWh

    # remove from memory
    model.terminate()
    del model
    return total_energy

In [ ]:
def cost_estimator(ts, base_cool_sp, base_heat_sp, mode, idf_steps_per_hour=6):

    if mode == 'cool':
        cool_sp = base_cool_sp + np.array([0.,-1.,1.])
        heat_sp = base_heat_sp + np.array([0., 0, 0])
    elif mode == 'heat':
        cool_sp = base_cool_sp + np.array([0., 0., 0])
        heat_sp = base_heat_sp + np.array([0., -1, 1.])
    
    energy = []
    for cool, heat in zip(cool_sp, heat_sp):
        energy.append(cost_of_comfort(ts, cool, heat, idf_steps_per_hour))
    
    return energy
    

In [ ]:
import collections 

#model = pyfmi.load_fmu('model.fmu')

ts = 0.0 
te = 86400.
#options = model.simulate_options()
idf_steps_per_hour = 6
dt = 3600./idf_steps_per_hour
ncp = (te - ts)/dt
#options['ncp'] = int(ncp)

# simulation setup
#model.setup_experiment(start_time=ts, stop_time=te)
#model.initialize()

# initialize output dic with empty list
res = collections.defaultdict(list)

# 
t = ts
while t < te:
    # setpoints
    csp = 24
    hsp = 20
    mode = 'heat'
    # cost of comfort
    costs = cost_estimator(t, csp, hsp, mode, idf_steps_per_hour)
    print(f'at time {t} for {mode}, keeping current setpoints, will cost {costs[0]} kWh energy in 30 days.')
    print(f'lowering setpoint by 1 C, will cost {costs[1]} kWh energy in 30 days.')
    print(f'increasing setpoint by 1 C, will cost {costs[2]} kWh energy in 30 days.')
    print('============================')
    # proceed forward
    #model.set('csp', csp)
    #model.set('hsp', hsp)
    #model.do_step(t, dt, new_step=True)
    #res['zone_temp'].append(model.get('zone_temp'))
    #res['csp'].append(model.get('csp'))
    #res['hsp'].append(model.get('hsp'))

    t = t + dt